In [16]:
import requests
import parsel
import time
from datetime import datetime
import re
import json
from tech_news.database import create_news, search_news, db, find_news

### Requisito 1

In [2]:
def fetch(url):
    time.sleep(1)
    try:
        response = requests.get(url=url, timeout=3)
        if response.ok:
            return response.text
        else:
            return None
    except requests.exceptions.ReadTimeout:
        return None

### Requisito 2

In [3]:
def scrape_novidades(html_content):
    selector = parsel.Selector(text=html_content)
    res = selector.css('.tec--list .tec--card__title__link')
    return list(map(lambda x: x.attrib['href'], res))

In [4]:
request_text = fetch('https://www.tecmundo.com.br/novidades')

In [5]:
scrape_novidades(request_text)

['https://www.tecmundo.com.br/internet/232452-google-translate-ios-recebe-traducoes-divididas-genero.htm',
 'https://www.tecmundo.com.br/minha-serie/232457-the-blacklist-9-temporada-episodio-9-recap.htm',
 'https://www.tecmundo.com.br/minha-serie/232439-demon-slayer-japao-fas-respeitam-personagens-maes.htm',
 'https://www.tecmundo.com.br/seguranca/232449-golpe-usando-token-falso-amazon-rouba-criptomoedas-vitimas.htm',
 'https://www.tecmundo.com.br/mercado/232464-momento-olhar-startups-nao-passou.htm',
 'https://www.tecmundo.com.br/minha-serie/232437-the-office-atrizes-serie-comentam-famoso-episodio-parkour.htm',
 'https://www.tecmundo.com.br/mercado/232458-microsoft-5-empresas-bilionarias-games-conhecer.htm',
 'https://www.tecmundo.com.br/mercado/232417-epic-games-volta-brigar-apple-devido-monopolio-ios.htm',
 'https://www.tecmundo.com.br/voxel/232477-ps4-pior-melhor-segundo-critica.htm',
 'https://www.tecmundo.com.br/voxel/232398-the-last-of-us-2-joel-esconde-detalhe-legal-flashback-e

### Requisito 03

In [6]:
def scrape_next_page_link(html_content):
    selector = parsel.Selector(text=html_content)
    return selector.css('.tec--list a.tec--btn::attr("href")').get()

### Requisito 4

In [7]:
def extract_count(strings_list, regex_r):
    output = 0
    for y in strings_list:
        try:
            regex = re.search(regex_r, y)
            return int(regex.groups()[0])
        except AttributeError:
            pass
    return output


def parse_json_writer(jsons_str):
    output = None
    for json_str in jsons_str:
        if json_str is None:
            return None

        try:
            deserialized_json = json.loads(json_str)
            output = deserialized_json["author"]["name"]
        except KeyError:
            pass

    return output


def extract_writer(selector):
    selectors = [
        selector.css(".tec--author__info__link::text").get(),
        selector.css(
            ".z--flex tec--timestamp tec--timestamp__item::text"
        ).get(),
        parse_json_writer(
            selector.css('script[type="application/ld+json"]::text').getall()
        ),
    ]

    for writer in selectors:
        if writer is not None:
            return str.strip(writer)

    return None


def scrape_noticia(html_content):
    selector = parsel.Selector(text=html_content)

    seletected = selector.css(".tec--toolbar__item *::text").getall()

    output = {
        "url": selector.css('link[rel="canonical"]::attr(href)').get(),
        "title": selector.css("#js-article-title::text").get(),
        "timestamp": selector.css("#js-article-date::attr(datetime)").get(),
        "writer": extract_writer(selector),
        "shares_count": extract_count(seletected, r"(\d+) Compartilharam"),
        "comments_count": extract_count(seletected, r"(\d+) Comentários"),
        "summary": "".join(
            selector.css(".tec--article__body > p:first-child ::text").getall()
        ),
        "sources": list(
            map(
                str.strip,
                selector.css(".z--mb-16 div a::text").getall(),
            )
        ),
        "categories": list(
            map(str.strip, selector.css("#js-categories a::text").getall())
        ),
    }
    return output


In [8]:
scrape_noticia(
  fetch('https://www.tecmundo.com.br/minha-serie/215285-falcao-soldado-invernal-fas-reacao-curiosa-sebastian-stan.htm')
)

{'url': 'https://www.tecmundo.com.br/minha-serie/215285-falcao-soldado-invernal-fas-reacao-curiosa-sebastian-stan.htm',
 'title': 'Falcão e o Soldado Invernal: fãs têm reação curiosa ao ver Sebastian Stan',
 'timestamp': '2021-04-09T20:00:01',
 'writer': 'Gabriel Witiuk',
 'shares_count': 0,
 'comments_count': 2,
 'summary': 'Atualmente fazendo sucesso na série Falcão e o Soldado Invernal, Sebastian Stan tem interpretado Bucky Barnes desde o primeiro filme do Capitão América. Stan contou que os fãs da Marvel sempre tentam "ativar" o lado assassino do personagem quando reconhecem o ator nas ruas.',
 'sources': ['CBR', 'MCU Exchange'],
 'categories': ['Minha Série', 'Séries', 'Elenco', 'Marvel', 'Disney+']}

### Requisito 5

In [31]:
def get_tech_news(n):
    url = 'https://www.tecmundo.com.br/novidades'
    links = []
    output = []

    while(len(links) < n):
      html_content = fetch(url)
      page_links = scrape_novidades(html_content)
      links.extend(page_links)
      url = scrape_next_page_link(html_content)

    for x in links[:n]:
      scrape = scrape_noticia(fetch(x))
      output.append(scrape)

    create_news(output)
    return output
      
get_tech_news(10);

In [10]:
def map_answer(news_list):
    return [(x["title"], x["url"]) for x in news_list]

### Requisito 6

In [11]:
def search_by_title(title):
    return map_answer(
        search_news({"title": {"$regex": title, "$options": "i"}})
    )

### Requisito 7

In [12]:
def parse_date(date):
    try:
        regex = re.search(r"(\d{4})-(\d{2})-(\d{2})", date)
        year, month, day = regex.groups()
        return datetime(int(year), int(month), int(day)).isoformat()
    except AttributeError:
        raise ValueError("Data inválida")
    except ValueError:
        raise ValueError("Data inválida")


def search_by_date(date):
    parse_date(date)
    return map_answer(search_news({"timestamp": {"$regex": date}}))

### Requisito 8

In [13]:
def search_by_source(source):
    return map_answer(
        search_news(
            {
                "sources": {
                    "$elemMatch": {"$regex": source, "$options": "i"}
                }
            }
        )
    )

### Requisito 9

In [14]:
def search_by_category(category):
    return map_answer(
        search_news(
            {
                "categories": {
                    "$elemMatch": {"$regex": category, "$options": "i"}
                }
            }
        )
    )

### Requisito 10

In [17]:
find_news()

[{'url': 'https://www.tecmundo.com.br/vamos.htm',
  'title': 'Vamoscomtudo',
  'timestamp': '2020-11-23T11:00:01',
  'writer': 'Leonardo',
  'shares_count': 1,
  'comments_count': 1,
  'summary': 'Sumario 2',
  'sources': ['ResetEra'],
  'categories': ['PC', 'Console']},
 {'url': 'https://www.tecmundo.com.br/internet/232452-google-translate-ios-recebe-traducoes-divididas-genero.htm',
  'title': 'Google Translate para iOS recebe traduções divididas por gênero',
  'timestamp': '2022-01-21T21:00:01',
  'writer': 'Iraci Falavina',
  'shares_count': 0,
  'comments_count': 0,
  'summary': 'A atualização do Google Translate que fornece traduções específicadas para cada gênero da palavra está finalmente chegando para os usuários do aplicativo no iOS. O Google anunciou este recurso para a versão web em 2018, e quatro anos depois ele está sendo implementado nos apps mobile.',
  'sources': ['XDA Developers'],
  'categories': ['Internet', 'Google', 'Google Tradutor']},
 {'url': 'https://www.tecmun

In [29]:
def top_5_news():
    set1 = {
      "$set": {
        "popularity": {
          "$add": ["$shares_count", "$comments_count"]
        }
      }
    }
    sort1 = {
      "$sort": {
        "popularity": -1,
        "title": 1
      }
    }
    limit1 = {
      "$limit": 5
    }

    return list(db.news.aggregate([set1, sort1, limit1]))

# top_5_news()

### Requsito 11

In [35]:
def top_5_categories():
    parcial_result = list(
        db.news.aggregate(
            [
                {"$unwind": "$categories"},
                {"$group": {"_id": "$categories", "total": {"$count": {}}}},
                {"$sort": {"total": -1, "_id": 1}},
                {"$limit": 5},
            ]
        )
    )
    return list(map(lambda x: x["_id"], parcial_result))

# top_5_categories()

['Mercado', 'Minha Série', 'Séries', 'Segurança', 'Amazon']